In [1]:
import pandas as pd
from vertexai.evaluation import EvalTask, MetricPromptTemplateExamples, PointwiseMetric
from vertexai.preview.evaluation import notebook_utils

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
LOCATION = "us-central1"  # @param {type:"string"}
EXPERIMENT = "rag-eval-01"  # @param {type:"string"}

import vertexai

vertexai.init(location=LOCATION)

In [3]:
questions = [
    "Which part of the brain does short-term memory seem to rely on?",
    "What provided the Roman senate with exuberance?",
    "What area did the Hasan-jalalians command?",
]

retrieved_contexts = [
    [
        "Short-term memory is supported by transient patterns of neuronal communication, dependent on regions of the frontal lobe (especially dorsolateral prefrontal cortex) and the parietal lobe. Long-term memory, on the other hand, is maintained by more stable and permanent changes in neural connections widely spread throughout the brain. The hippocampus is essential (for learning new information) to the consolidation of information from short-term to long-term memory, although it does not seem to store information itself. Without the hippocampus, new memories are unable to be stored into long-term memory, as learned from patient Henry Molaison after removal of both his hippocampi, and there will be a very short attention span. Furthermore, it may be involved in changing neural connections for a period of three months or more after the initial learning."
    ],
    [
        "In 62 BC, Pompey returned victorious from Asia. The Senate, elated by its successes against Catiline, refused to ratify the arrangements that Pompey had made. Pompey, in effect, became powerless. Thus, when Julius Caesar returned from a governorship in Spain in 61 BC, he found it easy to make an arrangement with Pompey. Caesar and Pompey, along with Crassus, established a private agreement, now known as the First Triumvirate. Under the agreement, Pompey's arrangements would be ratified. Caesar would be elected consul in 59 BC, and would then serve as governor of Gaul for five years. Crassus was promised a future consulship."
    ],
    [
        "The Seljuk Empire soon started to collapse. In the early 12th century, Armenian princes of the Zakarid noble family drove out the Seljuk Turks and established a semi-independent Armenian principality in Northern and Eastern Armenia, known as Zakarid Armenia, which lasted under the patronage of the Georgian Kingdom. The noble family of Orbelians shared control with the Zakarids in various parts of the country, especially in Syunik and Vayots Dzor, while the Armenian family of Hasan-Jalalians controlled provinces of Artsakh and Utik as the Kingdom of Artsakh."
    ],
]

generated_answers_by_rag_a = [
    "frontal lobe and the knee bone",
    "The Roman Senate was filled with exuberance due to successes against Catiline.",
    "The Hasan-Jalalians commanded the area of Syunik and Vayots Dzor.",
]

eval_dataset_rag_a = pd.DataFrame(
    {
        "user_input": questions,
        "retrieved_contexts": retrieved_contexts,
        "response": generated_answers_by_rag_a,
        "prompt": [
            "Answer the question: " + question + " Context: " + item[0]
            for question, item in zip(questions, retrieved_contexts)
        ],
    }
)

In [4]:
eval_dataset_rag_a

,user_input,retrieved_contexts,response,prompt
0,Which part of the brain does short-term memory...,[Short-term memory is supported by transient p...,frontal lobe and the knee bone,Answer the question: Which part of the brain d...
1,What provided the Roman senate with exuberance?,"[In 62 BC, Pompey returned victorious from Asi...",The Roman Senate was filled with exuberance du...,Answer the question: What provided the Roman s...
2,What area did the Hasan-jalalians command?,[The Seljuk Empire soon started to collapse. I...,The Hasan-Jalalians commanded the area of Syun...,Answer the question: What area did the Hasan-j...


In [5]:
from ragas.dataset_schema import SingleTurnSample
from ragas.metrics import ResponseGroundedness, Faithfulness
from ragas.llms import LangchainLLMWrapper
from langchain_google_vertexai import VertexAI


evaluator_llm = LangchainLLMWrapper(VertexAI(model_name="gemini-pro"))

sample = SingleTurnSample(
    user_input="What provided the Roman senate with exuberance?",
    response="The Roman Senate was filled with exuberance due to successes against Catiline.",
    retrieved_contexts=[
        "In 62 BC, Pompey returned victorious from Asia. The Senate, elated by its successes against Catiline, refused to ratify the arrangements that Pompey had made. Pompey, in effect, became powerless. Thus, when Julius Caesar returned from a governorship in Spain in 61 BC, he found it easy to make an arrangement with Pompey. Caesar and Pompey, along with Crassus, established a private agreement, now known as the First Triumvirate. Under the agreement, Pompey's arrangements would be ratified. Caesar would be elected consul in 59 BC, and would then serve as governor of Gaul for five years. Crassus was promised a future consulship."
    ],
)


scorer = ResponseGroundedness(llm=evaluator_llm)
faithfulness = Faithfulness(llm=evaluator_llm)

In [6]:
from vertexai.evaluation import CustomMetric, EvalTask, MetricPromptTemplateExamples
import asyncio
from vertexai.evaluation import constants

def run_ragas_faithfulness_metric(row_dict):
    # This function runs in a thread and executes the 
    # async function
    sample = SingleTurnSample(
                        user_input=row_dict.get("user_input"),
                        retrieved_contexts=row_dict.get("retrieved_contexts"),
                        response=row_dict.get("response"),
                        reference=row_dict.get("reference"),
                        rubric=row_dict.get("rubric"),
                    )
    return {"ragas_faithfulness" : asyncio.run(faithfulness.single_turn_ascore(sample=sample))}

custom_faithfulness = CustomMetric(
    name="ragas_faithfulness",
    metric_function=run_ragas_faithfulness_metric,
)

In [7]:
# monkey patching ragas evaluators
def hash_by_name(self):
    return hash(self.name)


ResponseGroundedness.__hash__ = hash_by_name
Faithfulness.__hash__ = hash_by_name

In [8]:
rag_eval_task_rag_a = EvalTask(
    dataset=eval_dataset_rag_a,
    metrics=[
        "question_answering_quality",
        scorer,
    ],
)

In [9]:
result_rag_a = rag_eval_task_rag_a.evaluate()

Computing metrics with a total of 3 Vertex Gen AI Evaluation Service API requests.


100%|██████████| 3/3 [00:03<00:00,  1.09s/it]


Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

All 3 metric requests are successfully computed.
Evaluation Took:9.043935417001194 seconds


In [10]:
result_rag_a.metrics_table

,user_input,retrieved_contexts,response,prompt,question_answering_quality/explanation,question_answering_quality/score,nv_response_groundedness/score
0,Which part of the brain does short-term memory...,[Short-term memory is supported by transient p...,frontal lobe and the knee bone,Answer the question: Which part of the brain d...,The response does not follow instructions very...,2.0,0.0
1,What provided the Roman senate with exuberance?,"[In 62 BC, Pompey returned victorious from Asi...",The Roman Senate was filled with exuberance du...,Answer the question: What provided the Roman s...,The response answers the question completely a...,5.0,1.0
2,What area did the Hasan-jalalians command?,[The Seljuk Empire soon started to collapse. I...,The Hasan-Jalalians commanded the area of Syun...,Answer the question: What area did the Hasan-j...,The response provides an incorrect answer acco...,1.0,0.0
